In [8]:
# Import required libraries
import os
import getpass
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken
import time
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load environment variables from .env file if available
load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [17]:
from openai import OpenAI
import json

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]


In [11]:
def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."



In [14]:
print(get_horoscope("rakesh"))# 2. Create a Deep Agent with SubAgent Delegation

rakesh: Next Tuesday you will befriend a baby otter.


In [15]:
# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]



In [18]:
# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_05cdc99a700323af00696b1e9eed248197ae7561887d5dd044', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_jn8GESX5mXG5kLKvxVwQw7Eo', name='get_horoscope', type='function_call', id='fc_05cdc99a700323af00696b1e9f97a08197aac8c042b22fc146', status='completed'), {'type': 'function_call_output', 'call_id': 'call_jn8GESX5mXG5kLKvxVwQw7Eo', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."}'}]
Final output:
{
  "id": "resp_05cdc99a700323af00696b1ea0c49c8197b475a7f03f1cecba",
  "created_at": 1768627872.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_05cdc

In [19]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    tools=[{"type": "web_search"}],
    input="What happened to India vs USA under 19 match yesterday?"
)

print(response.output_text)

If you mean the ICC U19 World Cup game on Thursday, January 15, 2026 (Bulawayo, Zimbabwe): India beat USA by 6 wickets (DLS). USA were bowled out for 107 in 35.2 overs; India’s target was revised to 96 and they reached 99/4 in 17.2 overs. Henil Patel took 5/16 and was Player of the Match; Abhigyan Kundu finished unbeaten on 42, sealing it with a six. ([espn.com](https://www.espn.com/cricket/series/8368/statistics/1511850/india-under-19s-vs-united-states-of-america-under-19s-1st-match-group-b-icc-mens-under-19-world-cup-2025-26?utm_source=openai))

If you were asking about a different match on January 16, tell me which competition/level and I’ll check.


In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
  model="gpt-5",
  reasoning={"effort": "low"},
  tools=[
      {
          "type": "web_search",
          "filters": {
              "allowed_domains": [
                  "pubmed.ncbi.nlm.nih.gov",
                  "clinicaltrials.gov",
                  "www.who.int",
                  "www.cdc.gov",
                  "www.fda.gov",
              ]
          },
      }
  ],
  tool_choice="auto",
  include=["web_search_call.action.sources"],
  input="Please perform a web search on how semaglutide is used in the treatment of diabetes.",
)

print(response.output_text)

In [20]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="o4-mini",
    tools=[{
        "type": "web_search",
        "user_location": {
            "type": "approximate",
            "country": "GB",
            "city": "London",
            "region": "London",
        }
    }],
    input="What are the best restaurants near me?",
)

print(response.output_text)

Here are eight standout restaurants across London, all highly rated in recent guides and well worth booking in advance:

• The Ledbury (Notting Hill) — A two-Michelin-star fine-dining staple led by Chef Brett Graham, celebrated for its refined tasting menus and global fusion dishes. Consistently ranked among London’s very best (Harden’s #4) and newly highlighted in Eater’s January 2026 roundup. ([timeout.com](https://www.timeout.com/london/news/38-london-restaurants-have-been-named-the-best-in-the-uk-in-a-prestigious-new-list-and-this-one-ranks-highest-120525?utm_source=openai))

• Core by Clare Smyth (Notting Hill) — Three-Michelin-starred restaurant from the chef behind Restaurant Gordon Ramsay, offering impeccably crafted modern British cuisine in an understated-luxury setting. ([timeout.com](https://www.timeout.com/london/news/38-london-restaurants-have-been-named-the-best-in-the-uk-in-a-prestigious-new-list-and-this-one-ranks-highest-120525?utm_source=openai))

• Ikoyi (Strand) — 

In [21]:
from openai import OpenAI
client = OpenAI()

resp = client.responses.create(
  model="gpt-5",
  tools=[{ "type": "web_search", "external_web_access": False }],
  tool_choice="auto",
  input="Find the sunrise time in Paris today and cite the source.",
)
print(resp.output_text)

Sunrise in Paris, France today (Saturday, January 17, 2026) is at 8:37 AM local time (CET, UTC+1). ([timeanddate.com](https://www.timeanddate.com/sun/france/paris?month=1))


In [ ]:
from openai import OpenAI

client = OpenAI()

instructions = """
You are a personal math tutor. When asked a math question,
write and run code using the python tool to answer the question.
"""

resp = client.responses.create(
    model="gpt-4.1",
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto", "memory_limit": "4g"}
        }
    ],
    instructions=instructions,
    input="I need to solve the equation 3x + 11 = 14. Can you help me?",
)

print(resp.output)

In [ ]:
from openai import OpenAI
client = OpenAI()

container = client.containers.create(name="test-container")

response = client.responses.create(
    model="gpt-4.1",
    tools=[{
        "type": "code_interpreter",
        "container": container.id
    }],
    tool_choice="required",
    input="use the python tool to calculate what is 4 * 3.82. and then find its square root and then find the square root of that result"
)

print(response.output_text)

In [23]:
client.containers.list()

SyncCursorPage[ContainerListResponse](data=[ContainerListResponse(id='cntr_696b26e550388190acf42d784f7bb8b00233aa7864cd324b', created_at=1768629989, name='test-container', object='container', status='running', expires_after=ExpiresAfter(anchor='last_active_at', minutes=20), last_active_at=1768630556, memory_limit='1g'), ContainerListResponse(id='cntr_696b266ad1b48190b3ddb564f4cf4990091082a71d61b59c', created_at=1768629866, name='auto', object='container', status='running', expires_after=ExpiresAfter(anchor='last_active_at', minutes=20), last_active_at=1768630556, memory_limit='4g'), ContainerListResponse(id='cntr_696b258762248190ad96c669024f93610b1155b8bb7e7339', created_at=1768629639, name='auto', object='container', status='running', expires_after=ExpiresAfter(anchor='last_active_at', minutes=20), last_active_at=1768630556, memory_limit='4g'), ContainerListResponse(id='cntr_696b2577c4788190a8f79ef65e06d1e80883ce169946aa25', created_at=1768629623, name='auto', object='container', stat

In [ ]:
#Files and images generated by the model are returned as annotations on the assistant's message. container_file_citation annotations point to files created in the container. They include the container_id, file_id, and filename. You can parse these annotations to surface download links or otherwise process the files.

# Iterate through the response to find file annotations
for item in response.output:
    if item.type == "message":
        for content in item.content:
            if hasattr(content, "annotations"):
                for annotation in content.annotations:
                    if annotation.type == "container_file_citation":
                        print(f"File generated: {annotation.container_file_citation.filename}")
                        # You can create a download link or process the file here using:
                        # annotation.container_file_citation.file_id


In [24]:
# MCP 

In [31]:
from openai import OpenAI

client = OpenAI()

resp = client.responses.create(
    model="gpt-5.2",
    tools=[
        {
            "type": "mcp",
            "server_label": "dmcp",
            "server_description": "A Dungeons and Dragons MCP server to assist with dice rolling.",
            "server_url": "https://dmcp-server.deno.dev/sse",
            "require_approval": "never",
        },
    ],
    input="Roll 2d16+1",
)

print(resp.output_text)

2d16 + 1 = **23**
